# BERT Fine-tune for Phishing URL Identification

[Reference Article: Fine-Tuning BERT for Text Classification by Shaw Talebi](https://towardsdatascience.com/fine-tuning-bert-for-text-classification-a01f89b179fc)

# Outline

1. Imports
2. Load training, test, and validation data
3. Load the pre-trained model and tokenizer
4. Load a binary classification head
5. Freeze all base model parameters
6. Unfreeze base model pooling layers
7. Tokenize data
8. Create data collator
9. Load evaluation metrics
10. Define hyperparameters
11. Create a trainer
12. Train the model
13. Validate the model

# 1. Imports

In [4]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

# 2. Load training, test, and validation data

In [5]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

Generating test split: 100%|██████████| 450/450 [00:00<00:00, 209575.48 examples/s]


# 3. Load the pre-trained model and tokenizer

In [7]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

# 4. Load a binary classification head

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5. Freeze all base model parameters

In [9]:
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# 6. Unfreeze base model pooling layers

In [10]:
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

# 7. Tokenize data

In [11]:
def preprocess_data(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset_dict.map(preprocess_data, batched=True)

Map: 100%|██████████| 450/450 [00:00<00:00, 17306.88 examples/s]


# 8. Create data collator

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 9. Load evaluation metrics

In [15]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # softmax to get probablities
    probablities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims = True)
    
    positive_class_probs = probablities[:, 1]
    
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)

    predicted_classes = np.argmax(predictions, axis=1)
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)
    
    return {"Accuracy": acc, "AUC": auc}
    

# 10. Define hyperparameters

In [16]:
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 11. Create a trainer

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 12. Train the model

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.496800,0.394079,0.809000,0.907000
2,0.402100,0.344547,0.842000,0.930000
3,0.375000,0.318336,0.871000,0.940000
4,0.347100,0.367796,0.849000,0.942000
5,0.346400,0.299696,0.869000,0.945000
6,0.335200,0.306380,0.860000,0.947000
7,0.323200,0.291947,0.876000,0.948000
8,0.321200,0.299626,0.858000,0.950000
9,0.330700,0.287512,0.871000,0.951000
10,0.321100,0.293075,0.864000,0.951000


TrainOutput(global_step=2630, training_loss=0.3598727730290518, metrics={'train_runtime': 205.0683, 'train_samples_per_second': 102.405, 'train_steps_per_second': 12.825, 'total_flos': 706603239165360.0, 'train_loss': 0.3598727730290518, 'epoch': 10.0})

# 13. Validate the model

In [20]:
predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.887), 'AUC': np.float64(0.947)}
